In [1]:
import contextlib
import os

import pandas as pd
from constants import max_steps, replicates

from mesa.batchrunner import batch_run_not_stupid
from mesa.examples.advanced.epstein_civil_violence.model import EpsteinCivilViolence

In [ ]:
identifier = 2
files = [f"data_{x:04}.csv" for x in range(3)]
files

['data_0001.csv', 'data_0002.csv']

In [ ]:
def in_file(filename: str):
    return f"inputs/{filename}"


def out_file(filename: str, identifier: int):
    return f"outputs/{identifier:04}_{filename}"

In [ ]:
def run_for_file(filename: str):
    data = pd.read_csv(in_file(filename), index_col=0)
    print(f"Loaded {len(data)} parameter samples.")

    result = batch_run_not_stupid(
        EpsteinCivilViolence,
        parameters=data,
        iterations=replicates,
        max_steps=max_steps,
        number_processes=None,
        display_progress=True,
    )

    # ripley_police = np.zeros((len(result), len(result[0]["ripley_l_police"])))
    # ripley_citizen = np.zeros((len(result), len(result[0]["ripley_l_citizen"])))
    # for nr, res in enumerate(result):
    #     ripley_police[nr] = res["ripley_l_police"]
    #     del res["ripley_l_police"]
    #     ripley_citizen[nr] = res["ripley_l_citizen"]
    #     del res["ripley_l_citizen"]

    result_df = pd.DataFrame(result)
    with contextlib.suppress(FileExistsError):
        os.mkdir("outputs")
    result_df.to_csv(out_file(filename, identifier))
    # np.save(out_file(f"ripley_police_{filename}"), ripley_police)
    # np.save(out_file(f"ripley_citizen_{filename}"), ripley_citizen)

In [5]:
for file in files:
    run_for_file(file)

Loaded 1024 parameter samples.


  0%|          | 0/10240 [00:00<?, ?it/s]

Process SpawnPoolWorker-12:
Process SpawnPoolWorker-8:
Process SpawnPoolWorker-10:
Process SpawnPoolWorker-11:
Process SpawnPoolWorker-6:
Process SpawnPoolWorker-7:
Process SpawnPoolWorker-5:
Process SpawnPoolWorker-4:
Process SpawnPoolWorker-1:
Process SpawnPoolWorker-2:
Process SpawnPoolWorker-3:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/wessel/.local/share/uv/python/cpython-3.13.4-linux-x86_64-gnu/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  File "/home/wessel/.local/share/uv/python/cpython-3.13.4-linux-x86_64-gnu/lib/python3.13/multiprocessing/process.py", line 313, in _bootstrap
    self.run()
    ~~~~~~~~^^
  

KeyboardInterrupt: 